Goal: pull out 1 buoy record that has turbidity and search for matching icesat2 granules

Order of operations:
- load info for 1-2 relevant stations


In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
from sliderule import sliderule, icesat2, earthdata

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



ModuleNotFoundError: No module named 'sliderule'

In [ ]:
# load a list of all the relevant ERDDAPs and their urls.
FF = pd.read_pickle("labeled_relevant_stations.pkl")
print(len(FF))
FF[(FF.buoy == True) & (FF.ssc == True) & (FF.chl == True)]
# URI_PP_SurfaceSonde	http://www.neracoos.org/erddap/

In [ ]:
e = ERDDAP(    
server= "https://erddap.bco-dmo.org/erddap/",    
protocol="tabledap",   
response="csv", )

e.dataset_id = "bcodmo_dataset_817952" 
e.variables = [    
"longitude",    
"latitude",    
"time",    
"POC" ] 
e.constraints = {    
"time>=": "2017-01-13T00:00:00Z",    
"time<=": "2017-01-16T23:59:59Z",}

#Print the URL - check 
url = e.get_download_url() 
print(url)

In [4]:


# "chl|turb|sedi|par|phot|secchi|ssc"
FF["buoy"] = False
FF["glider"] = False
FF["ship"] = False
FF["turb"] = False
FF["ssc"] = False
FF["phot"] = False
FF["chl"] = False
FF['bad'] = False
lib = {'buoy','glider','ship','turb','ssc','phot','chl'}


for jj in range(len(FF)):
    if jj % 100 ==0:
        print(jj)# give a printout every 100 for my sanity

    # make the info URL for this site for this time range
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv"
              )
    e.dataset_id = FF['sites'][jj]
    # e.constraints = {"time>=": min_time, "time<=": max_time}
    info_url = e.get_info_url()
    try:
        ds_info = pd.read_csv(info_url)# make a dataframe for all the metadata for this station
    except:
        FF.loc[jj,"bad"] = True
    else:
        for kwd in lib:
            idx = ds_info["Value"].str.contains(kwd, flags = re.IGNORECASE)
            if any(idx==True)==True:
                FF.loc[jj,kwd] = True

FF = FF[FF.bad == False] 
FF.reset_index(drop=True, inplace=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000


In [5]:
FF.to_pickle("labeled_relevant_stations.pkl")


In [ ]:
FF[(FF.buoy == True) & (FF.turb == True)]